Syntax natürlicher Sprachen, WS 2023/24

# 05 - Aufgabenblatt

In [1]:
import nltk
from nltk import Tree
from nltk import DependencyGraph
from spacy import displacy
from itertools import chain

In [2]:
def _tree_labeled(self, i):
    node = self.get_by_address(i)
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word + '(' + rel + ')', [self._tree_labeled(dep) for dep in deps]) if deps else word + '(' + rel + ')'

def tree_labeled(self):
    node = self.root
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled

def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split(); line_list.pop(0); line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))
    return "\n".join([i for i in sent_list[0:]])

def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())
    deps = [x for x in deps if x]
    ex, word_list, arc_list = [], [], []
    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1; head = int(dep[2]); label = dep[3]
        if head>line:
            start = index; end = head-1; direction = "left"
        else:
            start = head-1; end = index; direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})
    ex.append({"words": word_list,"arcs": arc_list})
    return ex

def show_dep_trees(sent_nr,style=1):
    sent = transform_nr_conll(sent_nr)
    dg = DependencyGraph(sent)
    if style == 0 or style == 2:
        tree_labeled = dg.tree_labeled()
        tree_labeled.pretty_print(unicodelines=True)   
    if style == 1 or style == 2:
        ex = displacy_dep_input(sent)
        html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

## Aufgabe 1 - Darstellung von Dependenz-Strukturen



#### Geben Sie in folgender Codezelle den entsprechenden Klammerausdruck der Dependenzstruktur des folgenden Satzes an und generieren Sie den entsprechenden Syntaxbaum, indem Sie die Codezelle anschließend ausführen.


In [ ]:
tree = Tree.fromstring("""
(ROOT
    Maria schenkte ihnen ein Buch aus dem Buchladen
)
""")

tree.pretty_print(unicodelines=True)

## Aufgabe 2 - Dependenzgrammatik

#### Schreiben Sie zu dem folgenden Satz eine Dependenzgrammatik, die die syntaktische Ambiguität der Satzanalyse erhält. (Verwenden Sie dabei die [*Primacy-of-Content-Words*](https://universaldependencies.org/u/overview/syntax.html#the-primacy-of-content-words)-Maxime des Universal-Dependencies-Schemas für die PP-Analyse).

In [4]:
sent = "I shot an elephant in my pajamas"

In [5]:
grammar = nltk.DependencyGrammar.fromstring("""
    'shot' -> 'I'
    
    """)

parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

no parse found for: I shot an elephant in my pajamas


## Aufgabe 3 - Manuelle Dependenzanalyse

#### Stellen Sie folgenden Satz im Dependenzmodell dar:
*Ich kenne einen älteren Bruder von zwei sehr lebhaften Mädchen.*

### a) Ergänzen Sie dafür zunächst den Konstituentenbaum mit Angabe der perkolierten (hochgereichten) Köpfe.

(Regeln für Baum:
`S=NP+VP`, `NP=DET+(ADJP)+N+(PP)` oder `NP=PRON` oder `NP=NUM+(ADJP)+N`, `PP=P+NP`, `ADJP=(ADV)+ADJ`, `VP=V+NP`)

#### Bestimmen Sie die Köpfe gemäß der UD *Primacy of Content Words*:
(`head(PP) = head(NP)`)

(`head(S) = head(VP)`)


In [ ]:
np1 = Tree(
    'NP ()',
    [Tree('PRON ()', ['Ich'])]
)
adjp2 = Tree (
 'ADJP ()',
    [Tree('ADV ()', ['sehr']), Tree('ADJ ()', ['lebhaften'])]
)
np3 = Tree(
'NP ()',
    [Tree('NUM ()', ['zwei'] ), adjp2, Tree('N ()',['Maedchen'] )]
)
pp = Tree(
    'PP ()',
    [Tree ('P ()', ['von']), np3]
)
adjp1 = Tree(
    'ADJP ()',
    [Tree('ADJ ()', ['aelteren'])]
)
np2 = Tree(
    'NP ()',
    [ Tree ('DET ()', ['einen']), adjp1, Tree ('N ()', ['Bruder']), pp]
)
vp = Tree(
    'VP ()',
    [Tree('V ()', ['kenne']), np2]
)
s = Tree(
    'S ()',
    [np1, vp]
)

s.pretty_print(unicodelines=True)

         S ()                                                                      
   ┌──────┴─────────────┐                                                           
   │                  VP ()                                                        
   │      ┌─────────────┴──────────────┐                                            
   │      │                          NP ()                                         
   │      │     ┌───────┬───────┬──────┴─────┐                                      
   │      │     │       │       │          PP ()                                   
   │      │     │       │       │      ┌─────┴──────┐                               
   │      │     │       │       │      │          NP ()                            
   │      │     │       │       │      │     ┌──────┴───────┬─────────────────┐     
 NP ()    │     │    ADJP ()    │      │     │           ADJP ()              │    
   │      │     │       │       │      │     │      ┌───────┴────────┐ 

### b) Fertigen Sie nun eine Tabelle an, in der Sie zu jedem Kopfwort seine Dependenten notieren. Für jede Gruppe unmittelbarer Konstituenten einer Phrase (Ko-Konstituenten) gilt dabei, dass die nicht-hochgereichten Köpfe die Dependenten des hochgereichten Kopfes sind (Nicht-Köpfe in der Phrase sind abhängig vom Kopf der Phrase).



|Kopf|Dependenten|
|----|-----------|


### c) Konvertieren Sie Ihre Tabelle nun in einen Dependenzbaum (Stemma).

In [6]:
dep_tree = Tree(
    "H",
    ["H1", "H2", "..."]
)

dep_tree.pretty_print()

     H     
  ___|___   
 H1  H2 ...



## Aufgabe 4 - Übergangsbasierter Shift-Reduce-Dependency-Parser

#### Gegeben sei folgender Dependenzgraph:

In [7]:
sent_nr = """
1 fährt 0 ROOT
2 ein 4 -
3 elektrisches 4 -
4 Auto 1 -
5 von 6 -
6 Tesla 4 -
7 schnell 1 - 
8 ? 1 -
"""

show_dep_trees(sent_nr)

#### Geben Sie  den Typ der REDUCE-Übergänge (`LEFTARC`, `RIGHTARC`) sowie die Reihenfolge deren Durchführung bei Verarbeitung dieser Struktur mit einem Shift-Reduce-Dependency-Parser an.


In [ ]:
sent_nr = """
1 fährt 0 ROOT
2 ein 4 -
3 elektrisches 4 -
4 Auto 1 -
5 von 6 -
6 Tesla 4 -
7 schnell 1 - 
8 ? 1 -
"""

show_dep_trees(sent_nr)